# Inference (De-identification)\n\nThis notebook runs the de-identification pipeline on a local input file.\n\nBy default the detector stack uses regex rules. To enable BERT/ONNX detectors, set the corresponding environment variables and ensure models exist locally.\n

In [ ]:
import os\nimport sys\nfrom pathlib import Path\n\n# AI_WAREHOUSE 3.0 cache layout (avoid $HOME/.cache)\nos.environ.setdefault('HF_HOME', '/mnt/c/ai_cache/huggingface')\nos.environ.setdefault('TRANSFORMERS_CACHE', os.environ['HF_HOME'])\nos.environ.setdefault('TORCH_HOME', '/mnt/c/ai_cache/torch')\nos.environ.setdefault('XDG_CACHE_HOME', '/mnt/c/ai_cache')\nos.environ.setdefault('PIP_CACHE_DIR', '/mnt/c/ai_cache/pip')\n\nfor key in ('HF_HOME', 'TORCH_HOME', 'XDG_CACHE_HOME', 'PIP_CACHE_DIR'):\n    Path(os.environ[key]).expanduser().mkdir(parents=True, exist_ok=True)\n\nsys.path.insert(0, str(Path.cwd() / 'src'))\n\nfrom deid_pipeline import DeidPipeline\n\ntmp = Path('test_input')\ntmp.mkdir(exist_ok=True)\npath = tmp / 'sample.txt'\npath.write_text('ID A123456789 and phone 0912345678.', encoding='utf-8')\n\nresult = DeidPipeline(language='zh').process(str(path), output_mode='replace')\nresult.to_dict()\n